In [ ]:
# Install packages needed for this chapter
install.packages(c("rbenchmark", "microbenchmark", "proftools"))
source("http://bioconductor.org/biocLite.R")
biocLite(c("graph", "Rgraphviz"))

# Measure one execution

In [1]:
# Measure one execution
system.time(runif(1e8))

# Measure multiple executions
library(rbenchmark)
bench1 <- benchmark(runif(1e8), replications=10)
bench1

# Mean execution time per replication
within(bench1, {
    elapsed.mean <- elapsed/replications
    user.self.mean <- user.self/replications
    sys.self.mean <- sys.self/replications
})

# Multiple samples of one execution
benchmark(runif(1e8), replications=rep.int(1, 10))

# Alternative, maybe more accurate, method
library(microbenchmark)
microbenchmark(runif(1e8), times=10)

   user  system elapsed 
   6.09    0.25    6.35 

ERROR: Error in library(rbenchmark): there is no package called 'rbenchmark'


# Compute sample variance of numeric vector x

In [2]:
# Compute sample variance of numeric vector x
sampvar <- function(x) {
    # Compute sum of vector x
    my.sum <- function(x) {
        sum <- 0
        for (i in x) {
            sum <- sum + i
        }
        sum
    }
    
    # Compute sum of squared variances of the elements of x from
    # the mean mu
    sq.var <- function(x, mu) {
        sum <- 0
        for (i in x) {
            sum <- sum + (i - mu) ^ 2
        }
        sum
    }
    
    mu <- my.sum(x) / length(x)
    sq <- sq.var(x, mu)
    sq / (length(x) - 1)
}

# Profile sampvar()
x <- runif(1e7)
Rprof("Rprof.out")
y <- sampvar(x)
Rprof(NULL)
summaryRprof("Rprof.out")

# Display profiling results graphically
library(proftools)
p <- readProfileData(filename="Rprof.out")
plotProfileCallGraph(p, style=google.style, score="total")

,self.time,self.pct,total.time,total.pct
"""sq.var""",0.76,63.33,0.76,63.33
"""my.sum""",0.42,35.00,0.42,35.00
"""findCenvVar""",0.02,1.67,0.02,1.67
,total.time,total.pct,self.time,self.pct
"""doTryCatch""",1.20,100.00,0.00,0.00
"""eval""",1.20,100.00,0.00,0.00
"""evaluate""",1.20,100.00,0.00,0.00
"""evaluate_call""",1.20,100.00,0.00,0.00
"""executor$execute""",1.20,100.00,0.00,0.00
"""handle""",1.20,100.00,0.00,0.00


ERROR: Error in library(proftools): there is no package called 'proftools'


# Profile memory using Rprof

In [3]:
# Profile memory using Rprof
Rprof("Rprof-mem.out", memory.profiling=TRUE)
# sampvar() is defined in previous example
y <- sampvar(x)
Rprof(NULL)
summaryRprof("Rprof-mem.out", memory="both")

# Profile using gcinfo()
gcinfo(TRUE)
y <- sampvar(x)
gcinfo(FALSE)

,self.time,self.pct,total.time,total.pct,mem.total
"""sq.var""",0.72,62.07,0.72,62.07,0
"""my.sum""",0.44,37.93,0.44,37.93,0
,total.time,total.pct,mem.total,self.time,self.pct
"""doTryCatch""",1.16,100.00,0,0.00,0.00
"""eval""",1.16,100.00,0,0.00,0.00
"""evaluate""",1.16,100.00,0,0.00,0.00
"""evaluate_call""",1.16,100.00,0,0.00,0.00
"""executor$execute""",1.16,100.00,0,0.00,0.00
"""handle""",1.16,100.00,0,0.00,0.00
"""handle_shell""",1.16,100.00,0,0.00,0.00


[1] FALSE

[1] TRUE

# Compute sample variance of numeric vector x

In [4]:
# Compute sample variance of numeric vector x
sampvar <- function(x) {
    # Compute sum of vector x
    my.sum <- function(x) {
        sum <- 0
        for (i in x) {
            sum <- sum + i
        }
        sum
    }
    
    mu <- my.sum(x) / length(x)
    sq <- my.sum((x - mu) ^ 2)
    sq / (length(x) - 1)
}

# Profile sampvar()
x <- runif(1e7)
Rprof("Rprof-mem.out", memory.profiling=TRUE)
y <- sampvar(x)
Rprof(NULL)
summaryRprof("Rprof-mem.out", memory="both")


,self.time,self.pct,total.time,total.pct,mem.total
"""my.sum""",0.92,97.87,0.92,97.87,76.8
"""grep""",0.02,2.13,0.02,2.13,0.0
,total.time,total.pct,mem.total,self.time,self.pct
"""doTryCatch""",0.94,100.00,76.8,0.00,0.00
"""eval""",0.94,100.00,76.8,0.00,0.00
"""evaluate""",0.94,100.00,76.8,0.00,0.00
"""evaluate_call""",0.94,100.00,76.8,0.00,0.00
"""executor$execute""",0.94,100.00,76.8,0.00,0.00
"""handle""",0.94,100.00,76.8,0.00,0.00
"""handle_shell""",0.94,100.00,76.8,0.00,0.00


# Compute sample variance of numeric vector x

In [5]:
# Compute sample variance of numeric vector x
sampvar <- function(x) {
    mu <- sum(x) / length(x)
    sq <- sum((x - mu) ^ 2)
    sq / (length(x) - 1)
}

# Profile sampvar()
x <- runif(1e7)
Rprof("Rprof-mem.out", memory.profiling=TRUE)
y <- sampvar(x)
Rprof(NULL)
summaryRprof("Rprof-mem.out", memory="both")

# Benchmark against var() using 100 repetitions
library(microbenchmark)
microbenchmark(sampvar(x), var(x))

,self.time,self.pct,total.time,total.pct,mem.total
"""-""",0.04,50,0.04,50,76.7
"""grepl""",0.02,25,0.02,25,0.1
"""invisible""",0.02,25,0.02,25,0.0
,total.time,total.pct,mem.total,self.time,self.pct
"""doTryCatch""",0.08,100,76.8,0.00,0
"""evaluate""",0.08,100,76.8,0.00,0
"""evaluate_call""",0.08,100,76.8,0.00,0
"""executor$execute""",0.08,100,76.8,0.00,0
"""handle_shell""",0.08,100,76.8,0.00,0
"""IRkernel::main""",0.08,100,76.8,0.00,0


ERROR: Error in library(microbenchmark): there is no package called 'microbenchmark'
